In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
!git clone https://github.com/hyeshik/colab-biolab.git
!cd colab-biolab && bash tools/setup.sh
exec(open('colab-biolab/tools/activate_conda.py').read())

fatal: destination path 'colab-biolab' already exists and is not an empty directory.
./
./root/
./root/.bin.priority/
./root/.bin.priority/pip
./root/.bin.priority/pip2
./root/.bin.priority/pip3
./root/.profile
./root/.bashrc.biolab
./root/.condarc
./root/.tmux.conf
./root/.vimrc
PREFIX=/root/conda
Unpacking payload ...

Installing base environment...

Preparing transaction: ...working... done
Executing transaction: ...working... done
installation finished.
    You currently have a PYTHONPATH environment variable set. This may cause
    unexpected behavior when running the Python interpreter in Miniconda3.
    For best results, please verify that your PYTHONPATH only points to
    directories of packages that are compatible with the Python interpreter
    in Miniconda3: /root/conda
Activated conda environment `lab'!


In [16]:
!conda install -y bedtools bioawk samtools

Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: - \ | / - done

## Package Plan ##

  environment location: /root/conda

  added / updated specs:
    - bedtools
    - bioawk
    - samtools


The following packages will be UPDATED:

  ca-certificates    pkgs/main::ca-certificates-2024.3.11-~ --> conda-forge::ca-certificates-2024.6.2-hbcca054_0 
  conda              pkgs/main::conda-24.4.0-py312h06a4308~ --> conda-forge::conda-24.5.0-py312h7900ff3_0 
  libgcc-ng          pkgs/main::libgcc-ng-11.2.0-h1234567_1 --> conda-forge::libgcc-ng-13.2.0-h77fa898_7 
  libgomp              pkgs/main::libgomp-11.2.0-h1234567_1 --> conda-forge::libgomp-13.2.0-h77fa898_7 
  libstdcxx-ng       pkgs/main::libstdcxx-ng-11.2.0-h12345~ --> conda-forge::libstdcxx-ng-13.2.0-hc0a3c3a_7 
  ncurses                 pkgs/main::ncurses-6.4-h6a678d5_0 --> conda-forge::ncurses-6.5-h59595ed_0 
  openssl              pkgs/main::openssl-3.0.13-h7f8727e_1 --> conda-forge:

In [23]:
%cd /content/drive/MyDrive/2024-01_class/bioinfo_class/binfo1-work

/content/drive/MyDrive/2024-01_class/bioinfo_class/binfo1-work


In [21]:
!pip install pysam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 10.1 MB/s eta 0:00:00


In [32]:
import pysam
import matplotlib.pyplot as plt
import numpy as np
from collections import defaultdict

In [37]:
# Function to read BAM file and extract error information
def extract_errors(file_path):
    bamfile = pysam.AlignmentFile(file_path, "rb")
    error_dict = defaultdict(lambda: {'substitution': 0, 'deletion': 0, 'insertion': 0})
    total_reads = 0

    for read in bamfile.fetch():
        total_reads += 1
        if not read.is_unmapped:
            query_sequence = read.query_sequence
            reference_sequence = read.get_reference_sequence()
            for (q_pos, ref_pos, cigar) in read.get_aligned_pairs(with_seq=True):
                if cigar is None:
                    continue
                if q_pos is None:  # Deletion
                    error_dict[ref_pos]['deletion'] += 1
                elif ref_pos is None:  # Insertion
                    error_dict[q_pos]['insertion'] += 1
                elif q_pos < len(query_sequence) and ref_pos < len(reference_sequence) and query_sequence[q_pos] != reference_sequence[ref_pos]:  # Substitution
                    error_dict[q_pos]['substitution'] += 1

    bamfile.close()
    return error_dict, total_reads

In [38]:
# Function to calculate error frequencies
def calculate_frequencies(error_dict, total_reads, read_length):
    positions = np.arange(read_length)
    substitutions = np.zeros(read_length)
    deletions = np.zeros(read_length)
    insertions = np.zeros(read_length)

    for pos, errors in error_dict.items():
        if pos < read_length:
            substitutions[pos] = (errors['substitution'] / total_reads) * 100
            deletions[pos] = (errors['deletion'] / total_reads) * 100
            insertions[pos] = (errors['insertion'] / total_reads) * 100

    return positions, substitutions, deletions, insertions

In [40]:
file_paths = [
    "CLIP-let7g.pileup",
    "CLIP-let7g-gene.pileup",
    "RPF-siLuc.bam",
    "RNA-control.bam",
    "RPF-silin28a.bam",
    "RNA-siLin28a.bam"
]

read_length = 100

In [ ]:
# Extract errors and calculate frequencies
plot_data = []
for file_path in file_paths[2:]:
    errors, total_reads = extract_errors(file_path)
    positions, substitutions, deletions, insertions = calculate_frequencies(errors, total_reads, read_length)
    plot_data.append((file_path, positions, substitutions, deletions, insertions))